In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import imageio
import os
import random
import numpy as np

In [2]:
# Cargar el mapa de México (archivo GeoJSON o Shapefile)
mapa_mexico = gpd.read_file("mexican-states.shp")

In [3]:
# Coordenadas de las ciudades
cities = {
    "Aguascalientes": (21.8853, -102.2916),
    "Mexicali": (32.6245, -115.4523),
    "La Paz": (24.1426, -110.3128),
    "San Francisco de Campeche": (19.8450, -90.5230),
    "Tuxtla Gutiérrez": (16.7528, -93.1156),
    "Chihuahua": (28.6353, -106.0889),
    "Ciudad de México": (19.4326, -99.1332),
    "Saltillo": (25.4232, -101.0053),
    "Colima": (19.2433, -103.7250),
    "Victoria de Durango": (24.0277, -104.6532),
    "Toluca": (19.2826, -99.6557),
    "Guanajuato": (21.0190, -101.2574),
    "Chilpancingo": (17.5514, -99.5006),
    "Pachuca": (20.1011, -98.7591),
    "Guadalajara": (20.6597, -103.3496),
    "Morelia": (19.7008, -101.1844),
    "Cuernavaca": (18.9261, -99.2308),
    "Tepic": (21.5067, -104.8940),
    "Monterrey": (25.6866, -100.3161),
    "Oaxaca de Juárez": (17.0732, -96.7266),
    "Puebla": (19.0414, -98.2063),
    "Santiago de Querétaro": (20.5888, -100.3899),
    "Chetumal": (18.5001, -88.2961),
    "San Luis Potosí": (22.1565, -100.9855),
    "Culiacán": (24.8091, -107.3940),
    "Hermosillo": (29.0729, -110.9559),
    "Villahermosa": (17.9895, -92.9475),
    "Ciudad Victoria": (23.7369, -99.1411),
    "Tlaxcala de Xicohténcatl": (19.3139, -98.2400),
    "Xalapa": (19.5423, -96.9100),
    "Mérida": (20.9674, -89.5926),
    "Zacatecas": (22.7709, -102.5833),
}

In [4]:
# Parámetros del costo
seller_hour_cost = 200  # MXN por hora
fuel_price_per_liter = 24.0  # MXN por litro
car_efficiency_km_per_l = 15  # km por litro
speed_kmh = 80  # Velocidad promedio del vendedor
toll_rate_per_km = 0.5  # Tarifa de peaje por km (MXN)

In [5]:
# Crear matriz de costos basada en distancia, tiempo, peajes y combustible
num_cities = len(cities)
city_names = list(cities.keys())
cost_matrix = np.zeros((num_cities, num_cities))

In [6]:
# Crear una carpeta para almacenar los frames del GIF
frames_folder = "aco_frames_geo"
os.makedirs(frames_folder, exist_ok=True)

In [7]:
# Convertir las coordenadas de las ciudades a un GeoDataFrame
city_points = gpd.GeoDataFrame({
    "City": list(cities.keys()),
    "geometry": [Point(lon, lat) for lat, lon in cities.values()]
}, crs="EPSG:4326")


In [8]:
def calculate_distance(coord1, coord2):
    from math import radians, sin, cos, sqrt, atan2
    R = 6371  # Radio de la Tierra en km
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Distancia en km


In [9]:
for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            distance = calculate_distance(cities[city_names[i]], cities[city_names[j]])
            travel_time = distance / speed_kmh
            cost_time = travel_time * seller_hour_cost
            cost_fuel = (distance / car_efficiency_km_per_l) * fuel_price_per_liter
            cost_toll = distance * toll_rate_per_km
            cost_matrix[i, j] = cost_time + cost_fuel + cost_toll

In [10]:
# Crear una clase de colonia de hormigas para generar frames
class AntColony:
    def __init__(self, cost_matrix, num_ants, evaporation_rate, alpha=1, beta=2):
        self.cost_matrix = cost_matrix
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.num_cities = cost_matrix.shape[0]
        self.pheromone_matrix = np.ones((self.num_cities, self.num_cities))

    def run_with_frames(self, num_iterations):
        best_cost = float('inf')
        best_route = None

        for iteration in range(num_iterations):
            all_routes = []
            all_costs = []

            for ant in range(self.num_ants):
                route = [random.randint(0, self.num_cities - 1)]
                for _ in range(self.num_cities - 1):
                    current_city = route[-1]
                    probabilities = self._calculate_probabilities(current_city, route)
                    next_city = np.random.choice(range(self.num_cities), p=probabilities)
                    route.append(next_city)

                route_cost = self._calculate_route_cost(route)
                all_routes.append(route)
                all_costs.append(route_cost)

                if route_cost < best_cost:
                    best_cost = route_cost
                    best_route = route

            self._update_pheromones(all_routes, all_costs)
            self._save_frame(best_route, iteration, best_cost)

        return best_route, best_cost

    def _calculate_probabilities(self, current_city, visited):
        pheromone = self.pheromone_matrix[current_city]
        heuristic = 1 / (self.cost_matrix[current_city] + 1e-10)
        probabilities = (pheromone ** self.alpha) * (heuristic ** self.beta)
        probabilities[list(visited)] = 0
        probabilities /= probabilities.sum()
        return probabilities

    def _calculate_route_cost(self, route):
        total_cost = sum(self.cost_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
        total_cost += self.cost_matrix[route[-1], route[0]]
        return total_cost

    def _update_pheromones(self, routes, costs):
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for route, cost in zip(routes, costs):
            for i in range(len(route) - 1):
                self.pheromone_matrix[route[i], route[i + 1]] += 1 / cost
            self.pheromone_matrix[route[-1], route[0]] += 1 / cost

    def _save_frame(self, route, iteration, cost):
        route_coords = [Point(cities[city_names[i]][1], cities[city_names[i]][0]) for i in route]
        route_line = gpd.GeoDataFrame({
            "geometry": [LineString(route_coords)]
        }, crs="EPSG:4326")

        fig, ax = plt.subplots(figsize=(12, 8))
        mapa_mexico.plot(ax=ax, color="lightgrey", edgecolor="black")
        city_points.plot(ax=ax, color="red", markersize=50)
        route_line.plot(ax=ax, color="blue", linewidth=2)

        for x, y, label in zip(city_points.geometry.x, city_points.geometry.y, city_points["City"]):
            ax.text(x, y, label, fontsize=8, ha="right", color="blue")

        plt.title(f"Iteración {iteration + 1} - Costo: {cost:.2f}")
        frame_path = os.path.join(frames_folder, f"frame_{iteration + 1}.png")
        plt.savefig(frame_path)
        plt.close()

In [11]:
# Ejecutar el algoritmo con generación de frames
aco = AntColony(cost_matrix, num_ants=50, evaporation_rate=0.5, alpha=1, beta=2)
best_route, best_cost = aco.run_with_frames(num_iterations=50)

In [12]:
# Crear el GIF animado
gif_path = "aco_route_evolution_geo.gif"
with imageio.get_writer(gif_path, mode="I", duration=1) as writer:
    for frame in sorted(os.listdir(frames_folder)):
        if frame.endswith(".png"):
            frame_path = os.path.join(frames_folder, frame)
            image = imageio.imread(frame_path)
            writer.append_data(image)

C:\Users\russb\AppData\Local\Temp\ipykernel_17188\1976265677.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_path)


In [13]:
print(f"GIF animado guardado en: {gif_path}")

# Mostrar la mejor ruta y su costo
print(f"Mejor Ruta: {best_route}")
print(f"Costo Total de la Mejor Ruta: {best_cost:.2f} MXN")

GIF animado guardado en: aco_route_evolution_geo.gif
Mejor Ruta: [22, 26, 4, 19, 12, 16, 6, 10, 21, 15, 11, 23, 0, 31, 14, 8, 17, 9, 24, 2, 25, 1, 5, 18, 7, 27, 13, 28, 20, 29, 30, 3]
Costo Total de la Mejor Ruta: 41402.67 MXN
